# Bowl_Type_Classification
A 3D Conv model that will be able to predict the 3 bowl types correctly. Please import the following libraries to get started.

In [168]:
!unzip /content/sample_data/test_set.zip -d /content/sample_data/test_set/
#!pip install scipy==1.1.0



Archive:  /content/sample_data/test_set.zip
   creating: /content/sample_data/test_set/test_set/
   creating: /content/sample_data/test_set/test_set/Fast0/
  inflating: /content/sample_data/test_set/test_set/Fast0/fs1_0.jpg  
  inflating: /content/sample_data/test_set/test_set/Fast0/fs1_1.jpg  
  inflating: /content/sample_data/test_set/test_set/Fast0/fs1_10.jpg  
  inflating: /content/sample_data/test_set/test_set/Fast0/fs1_11.jpg  
  inflating: /content/sample_data/test_set/test_set/Fast0/fs1_12.jpg  
  inflating: /content/sample_data/test_set/test_set/Fast0/fs1_13.jpg  
  inflating: /content/sample_data/test_set/test_set/Fast0/fs1_14.jpg  
  inflating: /content/sample_data/test_set/test_set/Fast0/fs1_15.jpg  
  inflating: /content/sample_data/test_set/test_set/Fast0/fs1_16.jpg  
  inflating: /content/sample_data/test_set/test_set/Fast0/fs1_17.jpg  
  inflating: /content/sample_data/test_set/test_set/Fast0/fs1_18.jpg  
  inflating: /content/sample_data/test_set/test_set/Fast0/fs1_19.

In [1]:
import numpy as np
import os
from scipy.misc import imread, imresize
import datetime
import os

We set the random seed so that the results don't vary drastically.

In [2]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.random.set_seed(30)

Using TensorFlow backend.
C:\Users\Administrator\Anaconda3\envs\conda\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Administrator\Anaconda3\envs\conda\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Administrator\Anaconda3\envs\conda\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Administrator\

AttributeError: module 'tensorflow' has no attribute 'random'

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [150]:
'''
train_doc = np.random.permutation(open('/notebooks/storage/Final_data/Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('/notebooks/storage/Final_data/Project_data/val.csv').readlines())
batch_size = 64
'''
train_doc = np.random.permutation(open('/content/sample_data/Dataset/train.csv').readlines())
val_doc = np.random.permutation(open('/content/sample_data/Dataset/val.csv').readlines())
batch_size = 5

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with some of the parts of the generator function such that you get high accuracy.

In [151]:
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [0,1,2,4,6,8,10,12,14,16,18,20,22,24,26,27,28,29]
    #img_idx = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29]
    while True:
        t = np.random.permutation(folder_list)
        num_batches = int(len(t)/batch_size)
        for batch in range(num_batches):
            batch_data = np.zeros((batch_size,18,128,128,3))
            batch_labels = np.zeros((batch_size,3))
            for folder in range(batch_size):
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0])
                for idx,item in enumerate(img_idx):
                    #print(imgs[item])
                    #print(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item])
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    image = imresize(image,(128,128)).astype(np.float32)
                    '''
                    if image.shape[1] == 160:
                        image = imresize(image[:,20:140,:],(84,84)).astype(np.float32)
                    else:
                        image = imresize(image,(84,84)).astype(np.float32)
                    '''
                    batch_data[folder,idx,:,:,0] = image[:,:,0] - 104
                    batch_data[folder,idx,:,:,1] = image[:,:,1] - 117
                    batch_data[folder,idx,:,:,2] = image[:,:,2] - 123
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels

        if (len(t)%batch_size) != 0:
            batch_data = np.zeros((len(t)%batch_size,18,128,128,3))
            batch_labels = np.zeros((len(t)%batch_size,3))
            for folder in range(len(t)%batch_size):
                imgs = os.listdir(source_path+'/'+ t[folder + (num_batches*batch_size)].split(';')[0])
                for idx,item in enumerate(img_idx):
                    image = imread(source_path+'/'+ t[folder + (num_batches*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    image = imresize(image,(128,128)).astype(np.float32)
                    '''
                    print(image.shape[1],image.shape[0])
                    if image.shape[1] == 460:
                        image = imresize(image[:,20:140,:],(84,84)).astype(np.float32)
                    else:
                        image = imresize(image,(84,84)).astype(np.float32)
                    '''

                    batch_data[folder,idx,:,:,0] = image[:,:,0] - 104
                    batch_data[folder,idx,:,:,1] = image[:,:,1] - 117
                    batch_data[folder,idx,:,:,2] = image[:,:,2] - 123

                batch_labels[folder, int(t[folder + (num_batches*batch_size)].strip().split(';')[2])] = 1

            yield batch_data, batch_labels

In [152]:
curr_dt_time = datetime.datetime.now()
'''
train_path = '/notebooks/storage/Final_data/Project_data/train'
val_path = '/notebooks/storage/Final_data/Project_data/val'
'''
train_path = '/content/sample_data/Dataset/train'
val_path = '/content/sample_data/Dataset/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 50
print ('# epochs =', num_epochs)

# training sequences = 62
# validation sequences = 30
# epochs = 50


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D`. Also remember that the last layer is the softmax. Remember that the network is designed in such a way that the model is able to fit in the memory of the webcam.

In [153]:
from keras.models import Sequential
from keras.layers import Dense, GRU, Dropout, Flatten, BatchNormalization, Activation
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers

model = Sequential()
model.add(Conv3D(64, (3,3,3), strides=(1,1,1), padding='same', input_shape=(18,128,128,3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(2,2,1), strides=(2,2,1)))

model.add(Conv3D(128, (3,3,3), strides=(1,1,1), padding='same'))

model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))

# model.add(Dropout(0.25))

model.add(Conv3D(256, (3,3,3), strides=(1,1,1), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))

# model.add(Dropout(0.25))

model.add(Conv3D(256, (3,3,3), strides=(1,1,1), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))

model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [154]:
'''
sgd = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.7, nesterov=True)
model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())
'''
"""
#Best so far
sgd = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.2, nesterov=True)
model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())
"""
sgd = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.2, nesterov=True)
model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_53 (Conv3D)           (None, 18, 128, 128, 64)  5248      
_________________________________________________________________
batch_normalization_53 (Batc (None, 18, 128, 128, 64)  256       
_________________________________________________________________
activation_53 (Activation)   (None, 18, 128, 128, 64)  0         
_________________________________________________________________
max_pooling3d_53 (MaxPooling (None, 9, 64, 128, 64)    0         
_________________________________________________________________
conv3d_54 (Conv3D)           (None, 9, 64, 128, 128)   221312    
_________________________________________________________________
batch_normalization_54 (Batc (None, 9, 64, 128, 128)   512       
_________________________________________________________________
activation_54 (Activation)   (None, 9, 64, 128, 128) 

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [155]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [156]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1, mode='min', epsilon=0.0001, cooldown=0, min_lr=0.00001)
callbacks_list = [checkpoint, LR]

/usr/local/lib/python3.6/dist-packages/keras/callbacks/callbacks.py:998: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


In [157]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [158]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/50
Source path =  /content/sample_data/Dataset/valSource path =  ; batch size = 5
 /content/sample_data/Dataset/train ; batch size = 5


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  app.launch_new_instance()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


11/13 [========================>.....] - ETA: 3s - loss: 7.8175 - categorical_accuracy: 0.3091

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:37: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:38: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


13/13 [==============================] - 21s 2s/step - loss: 7.9953 - categorical_accuracy: 0.2903 - val_loss: 16.9424 - val_categorical_accuracy: 0.3333

Epoch 00001: saving model to model_init_2020-07-1610_31_57.377954/model-00001-8.19134-0.29032-16.94237-0.33333.h5
Epoch 2/50
13/13 [==============================] - 16s 1s/step - loss: 5.6512 - categorical_accuracy: 0.3226 - val_loss: 1.0169 - val_categorical_accuracy: 0.3333

Epoch 00002: saving model to model_init_2020-07-1610_31_57.377954/model-00002-5.70782-0.32258-1.01692-0.33333.h5
Epoch 3/50
13/13 [==============================] - 16s 1s/step - loss: 2.2046 - categorical_accuracy: 0.3548 - val_loss: 2.2961 - val_categorical_accuracy: 0.3333

Epoch 00003: saving model to model_init_2020-07-1610_31_57.377954/model-00003-2.24147-0.35484-2.29613-0.33333.h5
Epoch 4/50
13/13 [==============================] - 16s 1s/step - loss: 2.0489 - categorical_accuracy: 0.3387 - val_loss: 1.0128 - val_categorical_accuracy: 0.3333

Epoch 0000

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [129]:
!rm -r /content/model_init_2020-07-1609_18_09.085398

In [176]:
#Test Code

from keras.models import load_model, Model
#######
    
model_name = '/content/model_init_2020-07-1610_31_57.377954/model-00025-0.05238-1.00000-0.19227-0.96667.h5'
    
#######
    
test_doc = open('/content/sample_data/test_set/test.csv').readlines()
test_path = '/content/sample_data/test_set/test_set'
num_test_sequences = len(test_doc)
batch_size=1
print ('# testing sequences =', num_test_sequences)
print ('# testing sequences =', batch_size)
test_generator = generator(test_path, test_doc, batch_size)
model = load_model(model_name)
print("Model loaded.")
model_func = Model(inputs=[model.input], outputs=[model.output])
    
acc = 0
num_batches = int(num_test_sequences/batch_size)
    
for i in range(num_batches):
    x,true_labels = test_generator.__next__()
    print ("shape of x:", x.shape, "and shape of true_labels:", true_labels.shape)
    pred_idx = np.argmax(model_func.predict_on_batch(x), axis=1)
    print("True_labels: ",true_labels)
    print("Pred_labels: ",pred_idx)
    for j,k in enumerate(pred_idx):
        if true_labels[j,k] == 1:
            acc += 1
                
if (num_test_sequences%batch_size) != 0:
    x,true_labels = test_generator.__next__()
    print ("shape of x:", x.shape, "and shape of true_labels:", true_labels.shape)
    pred_idx = np.argmax(model_func.predict_on_batch(x), axis=1)
    for j,k in enumerate(pred_idx):
        if true_labels[j,k] == 1:
            acc += 1

print('Accuracy is =', acc/num_test_sequences) 

# testing sequences = 7
# testing sequences = 1
Model loaded.
Source path =  /content/sample_data/test_set/test_set ; batch size = 1
shape of x: (1, 18, 128, 128, 3) and shape of true_labels: (1, 3)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  app.launch_new_instance()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


True_labels:  [[1. 0. 0.]]
Pred_labels:  [0]
shape of x: (1, 18, 128, 128, 3) and shape of true_labels: (1, 3)
True_labels:  [[0. 1. 0.]]
Pred_labels:  [0]
shape of x: (1, 18, 128, 128, 3) and shape of true_labels: (1, 3)
True_labels:  [[1. 0. 0.]]
Pred_labels:  [0]
shape of x: (1, 18, 128, 128, 3) and shape of true_labels: (1, 3)
True_labels:  [[1. 0. 0.]]
Pred_labels:  [0]
shape of x: (1, 18, 128, 128, 3) and shape of true_labels: (1, 3)
True_labels:  [[0. 0. 1.]]
Pred_labels:  [2]
shape of x: (1, 18, 128, 128, 3) and shape of true_labels: (1, 3)
True_labels:  [[0. 1. 0.]]
Pred_labels:  [1]
shape of x: (1, 18, 128, 128, 3) and shape of true_labels: (1, 3)
True_labels:  [[0. 1. 0.]]
Pred_labels:  [1]
Accuracy is = 0.8571428571428571


In [172]:
!rm -r /content/sample_data/test_set/test_set/Offspin1